In [14]:
import torchvision.transforms.v2 as v2
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [15]:
from torchvision.models import alexnet, AlexNet_Weights

In [16]:
import os
from pathlib import Path

my_transform = v2.Compose([ 

             v2.Resize((224, 224)), v2.ToTensor()])
dataset= ImageFolder('./080289/chap06/data/dogs-vs-cats', transform= my_transform)


data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [17]:
for X_train, y_label in data_loader:
    print(X_train.shape, y_label.shape)
    break

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [18]:
model=alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)

In [19]:
for p in model.parameters():
    p.requires_grad=False

In [20]:
 model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [21]:
for p in model.classifier.parameters():
    p.requires_grad=True

In [22]:
 model.classifier=nn.Sequential(
    nn.Dropout(),
    nn.Linear(256*6*6,512),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(512,64),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(64,2),
  
 )
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=512, bias=True)
  

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [24]:
def fit(model, data_loader, loss_fn, optimizer):
  for i in range(epochs):
    for X_train, y_label in data_loader:
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fn(outputs, y_label)
        loss.backward()
        optimizer.step()
    return model
       

In [25]:
pip install torch-summary

Note: you may need to restart the kernel to use updated packages.


In [26]:
epochs=1
for i in range(epochs):
    for X_train, y_label in data_loader:
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fn(outputs, y_label)
        loss.backward()
        optimizer.step()
    

In [27]:
EPOCHS = 1
for n in range(EPOCHS):
    print('EPOCH {}:'.format(n + 1))
    epoch_loss = 0
    step_loss = 0
    for idx, (X_train, y_label) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = loss_fn(outputs, y_label)
        loss.backward()
        optimizer.step()
        step_loss += loss.item()
        
        # 매 배치마다 손실값 출력
        print('  batch {} loss: {}'.format(idx + 1, step_loss))
        
    # 에포크 후 평균 손실 출력
    average_epoch_loss = step_loss / len(data_loader)
    print('  EPOCH {} average loss: {}'.format(n + 1, average_epoch_loss))

EPOCH 1:
  batch 1 loss: 0.5534573793411255
  batch 2 loss: 1.0313734114170074
  batch 3 loss: 1.507322520017624
  batch 4 loss: 2.0426049530506134
  batch 5 loss: 2.4804252088069916
  batch 6 loss: 2.9609545469284058
  batch 7 loss: 3.477757215499878
  batch 8 loss: 3.949516177177429
  batch 9 loss: 4.3815553188323975
  batch 10 loss: 4.9082284569740295
  batch 11 loss: 5.249964624643326
  batch 12 loss: 5.60647976398468
  batch 13 loss: 6.053019046783447
  batch 14 loss: 6.563012659549713
  batch 15 loss: 6.834055453538895
  batch 16 loss: 7.328967601060867
  EPOCH 1 average loss: 0.4580604750663042


In [29]:
from torchsummary import summary
summary(model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       (23,296)
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       (307,392)
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       (663,936)
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       (884,992)
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      (590,080)
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
├─Sequential: 1-3                        --
|    └─Dropout: 2-14                     --
|    └─Linear: 2-15                      4,719,104
|    └─ReLU: 2-16                        --
|    └─Dropout: 2-17                     --
|    └─Linear: 2-18           

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       (23,296)
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       (307,392)
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       (663,936)
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       (884,992)
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      (590,080)
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─AdaptiveAvgPool2d: 1-2                 --
├─Sequential: 1-3                        --
|    └─Dropout: 2-14                     --
|    └─Linear: 2-15                      4,719,104
|    └─ReLU: 2-16                        --
|    └─Dropout: 2-17                     --
|    └─Linear: 2-18           